In [1]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, TensorBoard
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from keras.preprocessing.image import img_to_array
from tqdm import tqdm
import math
import PIL

Using TensorFlow backend.


In [2]:
clean_data_path = '/home/evren/classified_data/clean_data/'
noisy_15dB_path = '/home/evren/classified_data/noisy_data/15dB/'


In [3]:
# Data Augmentation

datagen_clean = ImageDataGenerator(
    
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0,  # randomly shift images vertically (fraction of total height)
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.1,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,  
    vertical_flip=False, 
    rescale = 1./255,
    preprocessing_function=None,
    data_format='channels_last', # means that the images should have shape (samples, height, width, channels)
    validation_split=0.0,
    dtype='float32'
)

In [4]:
# Data Augmentation

datagen_noisy = ImageDataGenerator(
    
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0,  # randomly shift images vertically (fraction of total height)
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.1,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,  
    vertical_flip=False, 
    rescale = 1./255,
    preprocessing_function=None,
    data_format='channels_last', # means that the images should have shape (samples, height, width, channels)
    validation_split=0.0,
    dtype='float32'
)

In [5]:
clean_data_gen = datagen_clean.flow_from_directory(
                clean_data_path,
                target_size=(64,128),
                color_mode='grayscale',
                classes=None,
                class_mode='input', # "input" will be images identical to input images (mainly used to work with autoencoders)
                batch_size=128,
                shuffle=True, 
                seed=42,
                save_to_dir=None,
                save_prefix='augmented_',
                save_format='png',
                follow_links=False,
                subset=None,
                interpolation='nearest'
)

Found 69071 images belonging to 1 classes.


In [6]:
noisy_15dB_gen = datagen_noisy.flow_from_directory(
                noisy_15dB_path,
                target_size=(64,128),
                color_mode='grayscale',
                classes=None,
                class_mode='input', # "input" will be images identical to input images (mainly used to work with autoencoders)
                batch_size=128,
                shuffle=True, 
                seed=42,
                save_to_dir=None,
                save_prefix='augmented_',
                save_format='png',
                follow_links=False,
                subset=None,
                interpolation='nearest'
)

Found 69071 images belonging to 1 classes.


In [7]:
print(noisy_15dB_gen.batch_size)
print(clean_data_gen.batch_size)
steps_per_epoch = 69071 // noisy_15dB_gen.batch_size
print(steps_per_epoch)

128
128
539


In [8]:
data_list_15dB = []
batch_index = 1

while batch_index <= steps_per_epoch:
    data = noisy_15dB_gen.next()
    data_list_15dB.append(data[0])
    print(batch_index)    
    batch_index = batch_index + 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [9]:
data_array_15dB = np.asarray(data_list_15dB, dtype=np.float32)

print(len(data_list_15dB))
print(len(data_array_15dB))
del data_list_15dB

539
539


In [10]:
data_list_clean = []
batch_index = 1

while batch_index <= steps_per_epoch:
    data = clean_data_gen.next()
    data_list_clean.append(data[0])
    print(batch_index)    
    batch_index = batch_index + 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
data_array_clean = np.asarray(data_list_clean, dtype=np.float32)

print(len(data_list_clean))
print(len(data_array_clean))
del data_list_clean

539
539


SyntaxError: invalid syntax (<ipython-input-12-a5d5b61aa8a6>, line 1)

In [13]:
height = 64
width = 128
channel = 1


model = Sequential([
    
  # encoder
    Conv2D(8, (3,3), padding="same", activation="relu", input_shape=(height,width,channel)),
    BatchNormalization(),
    MaxPooling2D((2,2),strides=(2,2)),
    
    
    # decoder
    Conv2D(8, (3,3), padding="same", activation="relu"),
    UpSampling2D((2,2)),
    
    Conv2D(channel, (3,3), padding="same", activation="sigmoid") # sigmoid, relu    
])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 128, 8)        80        
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 128, 8)        32        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 64, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 64, 8)         584       
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 64, 128, 8)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 128, 1)        73        
Total params: 769
Trainable params: 753
Non-trainable params: 16
_________________________________________________________

In [15]:
model.compile(optimizer=Adam(lr=0.0001), loss='mse', metrics=['accuracy'])

In [16]:
history = model.fit(data_array_15dB,
                    epochs=15,
                   )

# If x is a dataset, generator, or keras.utils.Sequence instance, 
# y should not be specified (since targets will be obtained from x). 

ValueError: Error when checking input: expected conv2d_input to have 4 dimensions, but got array with shape (539, 128, 64, 128, 1)

In [ ]:
plt.rcParams['figure.figsize'] = (17,6)
fig, ax1 = plt.subplots()
xepochs = range(1,len(history.history['loss']) + 1)

plt.plot(xepochs, history.history['loss'], label = 'loss')
# plt.plot(xepochs, history.history['val_loss'], label = 'val_loss')
plt.title('Loss')
plt.legend()
plt.show()

plt.plot(xepochs, history.history['accuracy'], label = 'accuracy')
# plt.plot(xepochs, history.history['val_accuracy'], label = 'val_accuracy')
plt.title('Metrics (Accuracy)')
plt.legend()
plt.show()

In [ ]:
noisy_20dB_path = '/home/evren/classified_data/noisy_data/20dB/'

In [ ]:
noisy_20dB_gen = datagen_clean.flow_from_directory(
                noisy_20dB_path,
                target_size=(64,128),
                color_mode='grayscale',
                classes=None,
                class_mode='input', # "input" will be images identical to input images (mainly used to work with autoencoders)
                batch_size=69071,
                shuffle=True, 
                seed=42,
                save_to_dir=None,
                save_prefix='augmented_',
                save_format='png',
                follow_links=False,
                subset=None,
                interpolation='nearest'
)

In [ ]:
# kernel died while predicting

In [ ]:
decoded_20dB = model.predict(noisy_20dB_gen)
decoded_clean = model.predict(clean_data_gen)

In [ ]:
plt.figure(figsize=(17,4))
for i in range(1,7):
    ax = plt.subplot(1,7,i)
    plt.imshow(noisy_20dB_gen[i])
plt.show()

# Displaying prediction_train images
plt.figure(figsize=(17,4))
for i in range(1,7):
    ax = plt.subplot(1,7,i)
    plt.imshow(decoded_20dB[i])
plt.show()

In [ ]:
def PSNR(original, compressed): 
    mse = np.mean((original - compressed) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse)) 
    return psnr

In [ ]:
print(f'15dB PSNR mean: {PSNR(noisy_15dB, decoded_15dB)}')

for i in range(len(noisy_15dB)):
    
    value = PSNR(noisy_15dB[i], decoded_15dB[i]) 